In [1]:
!pip install gradio bypy huggingface_hub aligo

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.0/243.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
   ━━━

In [2]:
import os
import shutil
import huggingface_hub as hh
import pandas as pd

In [15]:
import concurrent.futures
import pandas as pd
from huggingface_hub import hf_hub_download
from huggingface_hub import HfApi # Import HfApi for list_repo_files, repo_info

def format_size(bytes, precision=2):
    """
    将文件大小从字节转换为可读格式（如 KB、MB、GB 等）。
    Huggingface 使用 1000 而非 1024。
    """
    units = ["B", "KB", "MB", "GB", "TB", "PB"]
    # Check if bytes is None or 0 before converting to float
    size = float(bytes) if bytes is not None and bytes != 0 else 0
    index = 0

    while size >= 1000 and index < len(units) - 1:
        index += 1
        size /= 1000

    return f"{size:.{precision}f} {units[index]}"

def list_repo_files_info(repo_id, token=None):
    """
    列出指定 Hugging Face 仓库中的文件信息。
    返回 DataFrame 和文件路径列表。
    """
    api = HfApi() # Instantiate HfApi
    data_ls = []
    for file in api.list_repo_files(repo_id=repo_id, repo_type="model"): # Call list_repo_files from HfApi instance
        # the `repo_info` function replaces `get_repo_file_metadata`
        # and is called differently
        repo_info_ = api.repo_info(repo_id=repo_id, revision=None, repo_type="model")
        # and the `size` attribute is now found under `blob` attribute
        file_size = next((blob.size for blob in repo_info_.siblings if blob.rfilename == file), None)
        data_ls.append([file, format_size(file_size)])
    files = [file[0] for file in data_ls]
    data = pd.DataFrame(data_ls, columns=['文件名', '大小'])
    return data, files


# 并行下载文件到当前目录下的"./download"目录
def download_file(repo_id, filenames):
    """
    并行下载 Hugging Face 仓库中的指定文件。
    """
    repo_name = repo_id.replace("/", "---")
    local_dir = f"./download/{repo_name}"

    def download_single_file(filename):
        """
        下载单个文件到指定目录。
        """
        print(f"开始下载：{filename}")
        return hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            local_dir=local_dir,
            local_dir_use_symlinks=False,
            force_download=True
        )

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # 提交所有文件下载任务
        futures = {executor.submit(download_single_file, filename): filename for filename in filenames}
        # 收集所有下载结果
        for future in concurrent.futures.as_completed(futures):
            filename = futures[future]
            try:
                result = future.result()
                print(f"文件下载完成：{filename} -> {result}")
            except Exception as e:
                print(f"文件下载失败：{filename}，错误信息：{e}")

    return local_dir

In [16]:
# 设置huggingface的repo_id
repo_id = "deepseek-ai/DeepSeek-V3"

In [17]:
# 查看huggingface的repo_id的文件列表，得到的filenames列表可以按需修改为自己想要下载的文件
data, filenames = list_repo_files_info(repo_id)
filenames

TypeError: float() argument must be a string or a real number, not 'NoneType'

In [ ]:
# 这里是下载到本地的路径
out_path = download_file(repo_id,filenames)

In [ ]:
!ls -l ./download/deepseek-ai-DeepSeek-V3

In [ ]:
from aligo import Aligo
refresh_token = "注意：改成自己的refresh_token"
ali = Aligo(refresh_token=refresh_token)

In [ ]:
user = ali.get_user()  # 获取用户信息
print(user.user_name, user.nick_name, user.phone)
ll = ali.get_file_list()  # 获取网盘根目录文件列表
ll

In [ ]:
# 在根目录下面先创建一个models目录
remote_folder = ali.get_folder_by_path('我的资源/models')
# 把下载路径上传到指定目录
ali.upload_folder(out_path)